<a href="https://colab.research.google.com/github/HanhaodiZhang/HATE/blob/main/augmentation_best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qq transformers

     |████████████████████████████████| 1.5MB 13.3MB/s 
     |████████████████████████████████| 2.9MB 50.4MB/s 
     |████████████████████████████████| 890kB 51.7MB/s 


In [3]:
import torch
import numpy as np
import pandas as pd

In [8]:
df = pd.read_csv("2.csv", index_col='id')

In [9]:
df_2 = pd.read_csv("olid-training.tsv", sep='\t', header=0, index_col='id')

In [10]:
df_2=df_2[['tweet','subtask_a']]

In [11]:
df = pd.concat( [df, df_2], axis=0 )

In [12]:
df

,tweet,subtask_a
id,,
864261.0,@USER She should ask a few native Americans wh...,OFF
864262.0,@USER She should whats a few native Americans ...,OFF
864263.0,@USER She should ask a few native Americans wh...,OFF
864261.0,@USER She should ask a few native Americans wh...,OFF
864262.0,@USER She should ask a few native Americans wh...,OFF
...,...,...
95338.0,@USER Sometimes I get strong vibes from people...,OFF
67210.0,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT
82921.0,@USER And why report this garbage. We don't g...,OFF


In [13]:
possible_labels=df.subtask_a.unique()
label_dict={}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label]=index

label_dict

{'NOT': 1, 'OFF': 0, nan: 2}

In [14]:
df['a_label']=df.subtask_a.replace(label_dict)
df.head()

,tweet,subtask_a,a_label
id,,,
864261.0,@USER She should ask a few native Americans wh...,OFF,0
864262.0,@USER She should whats a few native Americans ...,OFF,0
864263.0,@USER She should ask a few native Americans wh...,OFF,0
864261.0,@USER She should ask a few native Americans wh...,OFF,0
864262.0,@USER She should ask a few native Americans wh...,OFF,0


In [15]:
!pip install emoji --upgrade

     |████████████████████████████████| 51kB 6.0MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49717 sha256=f62cda5640f9d4261698feb23cfe80aba94eef9d322e1eb162ff6e7340363602
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [16]:
!pip install wordsegment

     |████████████████████████████████| 4.8MB 13.8MB/s 


In [17]:
from wordsegment import load
load()

In [20]:
from wordsegment import segment
import emoji
import re
def clean(word):
  
  text = re.sub(r"can't", "can not ", word)
  text = re.sub(r"don't", "can not ", text)
  text = re.sub(r"\'s", "", word)
  text = re.sub(r"\'ve", "have", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'m", "am", text)
  text = re.sub(r"%", "percent", text) 
  return text 

def foremoji(text):
  text = re.sub(r"’", "'", text)
  text = re.sub(r":", " : ", text)
  text=emoji.demojize(text)
  text = text.split()
  new_text=[]
  for w in text: 
    w=clean(w)  
    w=re.sub(r":", " <emoji> ", w)
    w=re.sub(r"@USER","",w)
    w=re.sub(r"_"," ",w)
    if re.match("^#",w):
      w=" ".join(segment(w))
    new_text.append(w)
  text = " ".join(new_text)
  return text

In [21]:
df['emoji_tweet']=df.tweet.apply(foremoji)

In [22]:
df

,tweet,subtask_a,a_label,emoji_tweet
id,,,,
864261.0,@USER She should ask a few native Americans wh...,OFF,0,She should ask a few native Americans what th...
864262.0,@USER She should whats a few native Americans ...,OFF,0,She should whats a few native Americans what ...
864263.0,@USER She should ask a few native Americans wh...,OFF,0,She should ask a few native Americans what th...
864261.0,@USER She should ask a few native Americans wh...,OFF,0,She should ask a few native Americans what th...
864262.0,@USER She should ask a few native Americans wh...,OFF,0,She should ask a few native Americans what th...
...,...,...,...,...
95338.0,@USER Sometimes I get strong vibes from people...,OFF,0,Sometimes I get strong vibes from people and ...
67210.0,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,1,Benidorm <emoji> white heavy check mark <emoj...
82921.0,@USER And why report this garbage. We don't g...,OFF,0,And why report this garbage. We don't give a ...


In [23]:
df['data_type']=['train']*df.shape[0]

In [24]:
from transformers import BertTokenizer

PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [25]:
from transformers import BertForSequenceClassification

model=BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME,
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [26]:
tokenizer.add_special_tokens({'additional_special_tokens':["<emoji>"]})
model.resize_token_embeddings(len(tokenizer))

Embedding(28997, 768)

In [29]:
dfval = pd.read_csv("olid-training.tsv", sep='\t', header=0, index_col='id')
dfval=dfval[['tweet','subtask_a']]
dfval.head()

,tweet,subtask_a
id,,
86426,@USER She should ask a few native Americans wh...,OFF
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
16820,Amazon is investigating Chinese employees who ...,NOT
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT


In [30]:
possible_labels=dfval.subtask_a.unique()
label_dict={}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label]=index

label_dict

{'NOT': 1, 'OFF': 0}

In [31]:
dfval['a_label']=dfval.subtask_a.replace(label_dict)
dfval.head()

,tweet,subtask_a,a_label
id,,,
86426,@USER She should ask a few native Americans wh...,OFF,0
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,0
16820,Amazon is investigating Chinese employees who ...,NOT,1
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,0
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,1


In [32]:
dfval['emoji_tweet']=dfval.tweet.apply(foremoji)

In [33]:
from sklearn.model_selection import train_test_split
RANDOM_SEED=17

X_train,X_val,y_train,y_val = train_test_split(
  dfval.index.values,
  dfval.a_label.values,
  test_size=0.5,
  random_state=RANDOM_SEED,
  # stratify=df.a_label.values
)

In [35]:
dfval['data_type']=['not_set']*dfval.shape[0]

dfval.loc[X_train,'data_type']='train'
dfval.loc[X_val,'data_type']='val'
dfval.head()

,tweet,subtask_a,a_label,emoji_tweet,data_type
id,,,,,
86426,@USER She should ask a few native Americans wh...,OFF,0,She should ask a few native Americans what th...,val
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,0,Go home you are drunk!!! maga trump2020 <e...,train
16820,Amazon is investigating Chinese employees who ...,NOT,1,Amazon is investigating Chinese employees who ...,train
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,0,"Someone shouldhaveTaken"" this piece of shit t...",train
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,1,Obama wanted liberals &amp; illegals to move...,train


In [36]:
encoder_data_train=tokenizer.batch_encode_plus(
    df[df.data_type=='train'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)

encoder_data_val=tokenizer.batch_encode_plus(
    dfval[dfval.data_type=='val'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation =True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)


In [39]:
inputs_ids_train=encoder_data_train['input_ids']
attentions_masks_train=encoder_data_train['attention_mask']
labels_train=torch.tensor(df[df.data_type=='train'].a_label.values)

inputs_ids_val=encoder_data_val['input_ids']
attentions_masks_val=encoder_data_val['attention_mask']
labels_val=torch.tensor(dfval[dfval.data_type=='val'].a_label.values)

In [40]:

from torch.utils.data import TensorDataset
dataset_train=TensorDataset(inputs_ids_train,
              attentions_masks_train, labels_train)
dataset_val=TensorDataset(inputs_ids_val,
              attentions_masks_val, labels_val)

In [41]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE=16

dataloader_train=DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=BATCH_SIZE
)
dataloader_val=DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=BATCH_SIZE
)

In [42]:

from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
epochs = 4

scheduler = get_linear_schedule_with_warmup(optimizer, 
                        num_warmup_steps=0,
                        num_training_steps=len(dataloader_train)*epochs)

In [43]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def classification_report_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return classification_report(labels_flat, preds_flat, digits=5)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [44]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':batch[0],
              'attention_mask':batch[1],
              'labels':batch[2],
              }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [45]:
epochs=6

In [46]:
from tqdm.notebook import tqdm
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)             
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.4145373283208638
Validation loss: 0.21415131893188913
F1 Score (Weighted): 0.9254470763392019



Epoch 2
Training loss: 0.193636653314111
Validation loss: 0.10772470386860811
F1 Score (Weighted): 0.9723181879633536



Epoch 3
Training loss: 0.10730704097752909
Validation loss: 0.06881998937772722
F1 Score (Weighted): 0.9829192754394838



Epoch 4
Training loss: 0.06830185854076531
Validation loss: 0.056331068183017606
F1 Score (Weighted): 0.9876013981978893



Epoch 5
Training loss: 0.05202032050204273
Validation loss: 0.05608126898335545
F1 Score (Weighted): 0.9876013981978893



Epoch 6
Training loss: 0.053735057660844084
Validation loss: 0.05608176497174798
F1 Score (Weighted): 0.9876013981978893



In [47]:
df_sweet = pd.read_csv("testset-levela.tsv", sep='\t', header=0, index_col='id')
df_sweet.head()

,tweet
id,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
27014,"#ConstitutionDay is revered by Conservatives, ..."
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...
13876,#Watching #Boomer getting the news that she is...
60133,#NoPasaran: Unity demo to oppose the far-right...


In [50]:
df_l = pd.read_csv("labels-levela.csv",header=None,names=['id','subtask_a'],index_col='id')

In [52]:
test = pd.merge(df_sweet, df_l, on=['id'])

In [53]:


possible_labels=test.subtask_a.unique()
label_dict={}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label]=index

label_dict

{'NOT': 1, 'OFF': 0}

In [57]:
test['data_type']=['test']*test.shape[0]

In [54]:
test['a_label']=test.subtask_a.replace(label_dict)

In [58]:
test['emoji_tweet']=test.tweet.apply(foremoji)

encoder_data_test=tokenizer.batch_encode_plus(
    test[test.data_type=='test'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)
inputs_ids_test=encoder_data_test['input_ids']
attentions_masks_test=encoder_data_test['attention_mask']
labels_test=torch.tensor(test[test.data_type=='test'].a_label.values)
dataset_test=TensorDataset(inputs_ids_test,
              attentions_masks_test, labels_test)
dataloader_test=DataLoader(
    dataset_test,
    sampler=RandomSampler(dataset_test),
    batch_size=4
)

In [71]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_1.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [72]:
_, predictions, true_test = evaluate(dataloader_test)

In [73]:
accuracy_per_class(predictions, true_test)

Class: OFF
Accuracy: 182/240

Class: NOT
Accuracy: 540/620



In [74]:
print(classification_report_func(predictions, true_test))

              precision    recall  f1-score   support

           0    0.69466   0.75833   0.72510       240
           1    0.90301   0.87097   0.88670       620

    accuracy                        0.83953       860
   macro avg    0.79883   0.81465   0.80590       860
weighted avg    0.84486   0.83953   0.84160       860

